# Communication, RS-232, ${\rm i^2c}$ and all that. 

## (Controlling an interface remotely with software).

Microcontrollers are great for collecting data, controller motors or power sources, but when it comes to managing very complex tasks, or large amounts of data they are not optimal. For isolated systems one solution to this problem is to store data to non-volitile memory (e.g., SD card) and recover the data later for analysis. However often we can connect the microcontroller to a more sophisticated system to manage and retrieve data in real time.

In order to accomplish this you need a) A communication channel of some kind and b) A protocol to send and receive messages. This week we'll explore a couple of different channels (direct wired serial communication, RS-232) and a form of network communication using WiFi (TCP/IP over WiFi). 

## RS-232

RS-232 is a three wire serial communication standard designed to provide easy transfer of data between two devices such as a computer and a "dumb terminal (keyboard/screen)" or between two computers. Each endpoint has a minimum of three wires: Transmit (Tx), Receive (Rx) and Ground (Gnd). There are some additional "handshaking" wires that can be used when one device is slower than the other and needs to indicate that it's not prepared for new communication (e.g., Data Set Ready (DTR), Ready To Send (RTS), etc.). The good news is that most devices can work well at useful speeds with only three wires. See Fig. 1 for a typical setup:


Project 5: Managing Data
=========================

Up to this point we've collected "bursts" of data quickly, then slowly transferred that data to our computers, after the fact, for anlysis. Sometimes we need to collect data over a long period of time and store and/or process that data on the computer as we're collecting it. This project provides an experience involving this situation.

Talking from the Arduino
------------------------

First consider the following Arduino program:

    /* Sample Program Illustrating Computer Interaction */

    #define LED 13          // pin 13 is the LED
    #define IN_PORT 0       // reading from analog in port 0
    #define NUMLOOPS 5      // every time we get a "g" command we take NUMLOOPS data points
    #define LOOP_DELAY 300  // How long to wait between measurements.

    void setup()
    {
        // start serial port at 9600 bps:
        Serial.begin(9600);
        Serial.println("Hello");  // tell the computer we're ready
        pinMode(LED, OUTPUT);     // set LED pin to output
    }

    void loop() {
      char command;
      int value;

      if (Serial.available()>0) {          // are there characters to read?
        command = (char)Serial.read();     // grab one
        if (command=='g') {                // check it
          Serial.println("Begin");         // it's a "g", let's go!
          for (int i=0; i<NUMLOOPS; i++) {
            value=analogRead(IN_PORT);     // read a data point
            delay(LOOP_DELAY);             // wait for some milliseconds
            Serial.print(i,DEC);           // print the count
            Serial.print(",");             // csv
            Serial.println(value, DEC);    // now the value, and a \cr
          }
          Serial.println("End");           // terminator
        }
      }
      else {
        digitalWrite(LED, HIGH);           // flash our light to let folks know we're waiting.
        delay(LOOP_DELAY);
        digitalWrite(LED, LOW);
        delay(LOOP_DELAY);
      }
    }
    
Let's look at this piece by piece:

    void setup()
    {
        // start serial port at 9600 bps:
        Serial.begin(9600);
        Serial.println("Hello");  // tell the computer we're ready
        pinMode(LED, OUTPUT);     // set LED pin to output
    }

As soon as the Arduino is reset it sets up the serial port to run at 9600 bits per second (this is a pretty standar bit rate for a serial port). Next, we send the signal "Hello" to let the computer know we're ready for commands. Also, set the LED pin as an output so we can let the rest of the world know we're waiting for input.

    void loop() {
      char command;
      int value;

      if (Serial.available()>0) {          // are there characters to read?
        command = (char)Serial.read();     // grab one
        if (command=='g') {                // check it
          Serial.println("Begin");         // it's a "g", let's go!
          for (int i=0; i<NUMLOOPS; i++) {
            value=analogRead(IN_PORT);     // read a data point
            delay(LOOP_DELAY);             // wait for some milliseconds
            Serial.print(i,DEC);           // print the count
            Serial.print(",");             // csv
            Serial.println(value, DEC);    // now the value, and a \cr
          }
          Serial.println("End");           // terminator
        }
      }

Here we first check to see if there are characters available in the serial port's buffer. Serial.available() returns the number of caracters in the buffer. Serial.read() grabs a single character and returns it to the program. We're going to check for a particular character ('g' in this case) and if we see that, we'll do something. In this case we first send "Begin", then enter into a loop that reads voltages on analog pin 0, sends them to the computer, and then finally we send "End".

It's easy to check this program in the "Serial Monitor" of the Arduino program editor. Try it!

Python on the Computer
----------------------

Next we need to write a program on the computer to read the data and store it. One of the easiest languages for such work is python: <http://www.python.org>. If you don't already have python on your oomputer you should install it. I would recommend one of the pre-built distributions like anaconda: <https://store.continuum.io/cshop/anaconda/>. For the purposes of this project you'll need to install a serial port module like pyserial. You can install pyserial by using the "pip" command:

    pip install pyserial
    
Consider the following python program (see the comments preceeded by '#' characters):






In [ ]:
#
# talk to Arduino!
#

import serial

#port = serial.Serial('COM3')                   # open the serial port (WIN)
port = serial.Serial('/dev/cu.usbmodem1A1211')  # open the serial port (MAC/Linux)

print("Waiting for conversation...")

s = port.readline().strip()             # read a line of input from the Arduino

print ("got ",str(s))                   # echo to screen

while s != 'Hello':                     # check for "Hello"
    s = port.readline().strip()         # not yet, try again
    print ("got ",str(s))

port.write('g')                         # send a command

s = port.readline().strip()             # get a reply

while s[:3] != 'End':                   # is it the end?
    print(str(s))                       # nope, echo, repeat
    s = port.readline().strip()
    
print ("Finally got:", str(s))          # that's it!
print ("Finished!")

port.close()                            # shut it down



if we create this program (call it "proj5.py") and run it from the command line we see:

    > python proj5.py
    Waiting for conversation...
    got  Hello
    Begin
    0,464
    1,423
    2,405
    3,391
    4,381
    Finally got: End
    Finished!

It's hard to get much simpler than that! 

# Communicating with $i^2c$

In order to permit a large number of devices with a small number of "pins" $i^2c$ was developed [in 1982](https://en.wikipedia.org/wiki/I²C). Basically the "serial bus" system uses two wires (SCL [clock], SDA [data]) which are never driven "high", but only ever pulled "low" by systems sharing the "bus". The advantage of this is that there is never any risk of two devices "fighting" over control in a way that could result in shorts or power being delivered from one device to another. Each wire is "pulled high" by a resistor (usually $5 k\Omega$) or so).

The easiest way to access $i^2c$ peripherals on the Arduino is to use the ["Wire"](https://www.arduino.cc/en/reference/wire) library (see the example link below). 

[i2c tutorial](https://learn.sparkfun.com/tutorials/i2c "An $i^2c$ Tutorial")

[example](https://howtomechatronics.com/tutorials/arduino/how-i2c-communication-works-and-how-to-use-it-with-arduino/)

Deliverables
------------

Your objective is to control the Arduino over the serial port to measure some kind of interesting value over a period of one or two minutes (maybe sampling once every  second or two? You should have *tens* of data points, or maybe a hundred or so at the *most*. No need for thousands of data values.) Do something to make your data vary a little during the "experiment". Please use at least one $i^2c$ peripheral in this experiment. One option is to measure temperature using an $i^2c$ temperature probel, or some other $i^2c$ based breakout board (e.g., altitude, humidity, light level), or you could use a probe that produces a voltage, but store the data in an $i^2c$ based memory. You could also create a program that controlled something (e.g., a motor, switch, etc.) using $i^2c$. Be creative! 

You should produce a standard report. Be sure to explain what you measured and how you determined that it was "working".